In [101]:
import requests                     # This is used to make HTTP requests
from selenium import webdriver      # This performs actions in the browser
from bs4 import BeautifulSoup       # This is used to parse the HTML

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Set up the headless Chrome options
chrome_options = Options()                          # Create the options instance
chrome_options.add_argument('--headless')           # No visible browser window
chrome_options.add_argument('--disable-gpu')        # Disable the GPU to speed things up

# Set up the Chrome driver 
driver = webdriver.Chrome(options=chrome_options)

# Load the Speeches page
driver.get('https://miniszterelnok.hu/beszedek/')

In [3]:
while True:
    try:
        show_more_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="interjuk"]/div/div[3]/a')))
        show_more_button.click()
    except:
        break

In [6]:
# Get the page source of the current page
page_source = driver.page_source

# Create a BeautifulSoup object from the page source
soup_main = BeautifulSoup(page_source, 'html.parser')

# Find all the title links on the main page and extract their URLs
title_links = soup_main.select('h3 a')
title_urls = list(set([link['href'] for link in title_links]))  # For some reason there are duplicates

In [7]:
title_urls

['https://miniszterelnok.hu/orban-viktor-napirend-elotti-felszolalasa-2023-02-27/',
 'https://miniszterelnok.hu/orban-viktor-facebook-nyilatkozata-az-europai-tanacs-uleset-kovetoen/',
 'https://miniszterelnok.hu/orban-viktor-facebook-nyilakozata-a-nyugdijemelesrol-rendelkezo-kormanyrendelet-alairasat-kovetoen/',
 'https://miniszterelnok.hu/2023-02-10-orban-viktor-facebook-nyilakozata-az-europai-tanacs-uleset-kovetoen/',
 'https://miniszterelnok.hu/orban-viktor-viszonvalasza-2023-02-27/',
 'https://miniszterelnok.hu/orban-viktor-nyilatkozata-ilham-alijev-azerbajdzsan-elnokevel-tortent-targyalasat-kovetoen/',
 'https://miniszterelnok.hu/orban-viktor-sajtonyilatkozata-robert-golobbal-szlovenia-miniszterelnokevel-tortent-talalkozojat-kovetoen/',
 'https://miniszterelnok.hu/orban-viktor-nyilatkozata-a-zoldenergia-fejlesztesi-es-szallitasi-egyuttmukodesi-megallapodas-unnepelyes-alairasat-megelozoen/',
 'https://miniszterelnok.hu/orban-viktor-sajtonyilatkozata-abdel-fattah-el-sziszivel-az-egy

In [92]:
speech_dates = []
speech_sources = []
speech_titles = []
speech_texts = []

for url in title_urls:
    # Scrape the first URL with beautiful soup
    soup = BeautifulSoup(requests.get(url).text, 'html.parser')

    # Find the date of the speech
    date_regex = re.compile(r'\d{4}\.\d{2}\.\d{2}')
    speech_date = soup.find(text=date_regex).strip().replace('.', '-')
    speech_dates.append(speech_date)

    # Find source of the speech
    forras_regex = re.compile(r'Forrás:*')
    speech_source = soup.find(text=forras_regex).strip().replace('Forrás: ', '')
    speech_sources.append(speech_source)

    # Find the title of the speech
    speech_title = soup.find('h1').text
    speech_titles.append(speech_title)

    # Find the text of the speech
    speech_text = ''
    # Loop through all p elements
    for p in soup.find_all('p'):
        # If the p element has a class attribute, skip it
        if p.has_attr('class'):
            continue
        # Otherwise, add the text to the speech_text variable
        else:
            speech_text += p.text + ' '

    remove_beginning = 'Beszédek / '
    remove_end = 'orbanviktor@orbanviktor.huOrbán Viktor1357 Budapest, Pf. 1'

    speech_text = speech_text.replace(remove_beginning, '')
    speech_text = speech_text.replace(speech_title, '')
    speech_text = speech_text.replace(remove_end, '')
    speech_text = speech_text.strip()

    speech_texts.append(speech_text)

/var/folders/x2/6gs4vmmn5bxgqplmvj03kx240000gn/T/ipykernel_2433/2762827269.py:12: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  speech_date = soup.find(text=date_regex).strip().replace('.', '-')
/var/folders/x2/6gs4vmmn5bxgqplmvj03kx240000gn/T/ipykernel_2433/2762827269.py:17: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  speech_source = soup.find(text=forras_regex).strip().replace('Forrás: ', '')


In [93]:
print(speech_dates)
print(speech_sources)
print(speech_titles)
print(speech_texts)

['2023-02-27', '2022-12-15', '2022-12-13', '2023-02-10', '2023-02-27', '2023-01-30', '2022-12-02', '2022-12-17', '2023-02-28', '2023-03-16', '2023-01-30', '2023-02-01', '2022-12-21', '2022-12-06', '2023-03-09', '2023-02-18', '2023-02-09']
['Miniszterelnöki Kabinetiroda', 'Miniszterelnöki Kabinetiroda', 'Miniszterelnöki Kabinetiroda', 'Miniszterelnöki Kabinetiroda', 'Miniszterelnöki Kabinetiroda', 'Miniszterelnöki Kabinetiroda', 'Miniszterelnöki Kabinetiroda', 'Miniszterelnöki Kabinetiroda', 'Miniszterelnöki Kabinetiroda', 'MTI', 'Miniszterelnöki Kabinetiroda', 'Miniszterelnöki Kabinetiroda', 'Miniszterelnöki Kabinetiroda', 'MTI', 'Miniszterelnöki Kabinetiroda', 'Miniszterelnöki Kabinetiroda', 'Miniszterelnöki Kabinetiroda']
['Orbán Viktor napirend előtti felszólalása', 'Orbán Viktor Facebook-nyilatkozata az Európai Tanács ülését követően', 'Orbán Viktor Facebook-nyilakozata a nyugdíjemelésről rendelkező kormányrendelet aláírását követően', 'Orbán Viktor Facebook-nyilakozata az Európai 

In [95]:
df = pd.DataFrame({'date': speech_dates, 'source': speech_sources, 'title': speech_titles, 'text': speech_texts})

In [98]:
df.head(16)

,date,source,title,text
0,2023-02-27,Miniszterelnöki Kabinetiroda,Orbán Viktor napirend előtti felszólalása,Tisztelt Elnök Úr! Tisztelt Országgyűlés! Tisz...
1,2022-12-15,Miniszterelnöki Kabinetiroda,Orbán Viktor Facebook-nyilatkozata az Európai ...,"Ezt is megéltük. Itt vagyunk Brüsszelben, az E..."
2,2022-12-13,Miniszterelnöki Kabinetiroda,Orbán Viktor Facebook-nyilakozata a nyugdíjeme...,"Most írtam alá azt a kormányrendeletet, amelly..."
3,2023-02-10,Miniszterelnöki Kabinetiroda,Orbán Viktor Facebook-nyilakozata az Európai T...,"Ma hajnalban, 3 órakor ért véget az európai mi..."
4,2023-02-27,Miniszterelnöki Kabinetiroda,Orbán Viktor viszonválasza,"Köszönöm szépen, Elnök úr! A bőség zavarával k..."
5,2023-01-30,Miniszterelnöki Kabinetiroda,"Orbán Viktor nyilatkozata Ilham Alijev, Azerba...","Jó napot kívánok, tisztelt Hölgyeim és Uraim! ..."
6,2022-12-02,Miniszterelnöki Kabinetiroda,Orbán Viktor sajtónyilatkozata Robert Golobbal...,Tisztelettel köszöntöm Önöket! Jó napot kíváno...
7,2022-12-17,Miniszterelnöki Kabinetiroda,Orbán Viktor nyilatkozata a zöldenergia-fejles...,Köszönöm szépen! Mélyen tisztelt Elnök Asszony...
8,2023-02-28,Miniszterelnöki Kabinetiroda,Orbán Viktor sajtónyilatkozata Abdel Fattah el...,"Köszönjük Szíszi elnök úrnak, hogy vendégül lá..."
9,2023-03-16,MTI,Orbán Viktor felszólalása a Türk Államok Szerv...,Mélyen tisztelt Erdoğan Elnök Úr! Tisztelt Eln...


In [99]:
# Save the df as pickle
df.to_pickle('speeches.pkl')